In [30]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import glob
import cv2
import mmdet
from mmdet.apis import init_detector, inference_detector
from mmdet.datasets import replace_ImageToTensor, build_dataset, build_dataloader
from mmdet.datasets.pipelines import Compose
from mmdet.core import bbox_cxcywh_to_xyxy, bbox_xyxy_to_cxcywh, bbox_xyxy_to_cxcywh
from tqdm import tqdm
import pickle
from mmdet.core.bbox.iou_calculators.iou2d_calculator import BboxOverlaps2D
import lap
import json
plt.rcParams['figure.dpi'] = 100

In [31]:
def _get_ece(preds, targets, n_bins=15):
    """
    ECE ported from Asukha et al., 2020.
    :param preds: Prediction probabilities in a Numpy array
    :param targets: Targets in a numpy array
    :param n_bins: Total number of bins to use.
    :return: Expected calibration error.
    """
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]

    confidences, predictions = np.max(preds, 1), np.argmax(preds, 1)
    accuracies = (predictions == targets)

    ece = 0.0
    avg_confs_in_bins = []
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        in_bin = np.logical_and(confidences > bin_lower, confidences <= bin_upper)
        prop_in_bin = np.mean(in_bin)
        if prop_in_bin > 0:
            accuracy_in_bin = np.mean(accuracies[in_bin])
            avg_confidence_in_bin = np.mean(confidences[in_bin])
            delta = avg_confidence_in_bin - accuracy_in_bin
            avg_confs_in_bins.append(delta)
            ece += np.abs(delta) * prop_in_bin
        else:
            avg_confs_in_bins.append(None)
    # For reliability diagrams, also need to return these:
    # return ece, bin_lowers, avg_confs_in_bins
    return ece

In [32]:
def linear_assignment(cost_matrix):
    _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
    matches = np.array([[y[i], i] for i in x if i >= 0])
    sort_idx = np.argsort(matches[:, 1])
    matches = matches[sort_idx]
    matches = torch.from_numpy(matches).long()
    return matches

In [33]:
with open('../../logs/detr_r50_4x16_decoder_and_output/merged_outputs_075_minimal.json') as f:
    merged_outputs = json.load(f)

# with open('../../logs/detr_r50_8x2_150e_coco/merged_outputs_075_minimal.json') as f:
#     merged_outputs = json.load(f)
    
with open('../../logs/detr_r50_4x16_decoder_and_output/output_model_1.pkl', 'rb') as f:
    model_output = pickle.load(f)
    

fnames = [o['ori_filename'] for o in model_output]

In [45]:
dir(merged_outputs['000000397133.jpg'][0])

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [25]:
all_probs, all_labels = [], []
count_unmatched = 0
count_no_preds = 0
total_gt_bboxes = 0
unmatched_gt_boxes = []
IOU_THRESHOLD = 0.5

for sample in model_output:
    fname = sample['ori_filename']
    gt_bboxes = torch.from_numpy(sample['gt_bboxes'])
    gt_labels = torch.from_numpy(sample['gt_labels']).long()    
    
    total_gt_bboxes += len(gt_labels)
    
    if merged_outputs[fname] == []:
        count_no_preds += 1
        all_labels.append(gt_labels)
        all_probs.append(torch.ones(len(gt_labels), 81)/81.)
        continue
    
    preds = merged_outputs[fname][0]
    preds = torch.tensor(preds)
    if len(preds) == 0:
        count_no_preds += 1
        #print(preds, fname, len(gt_labels))
        all_labels.append(gt_labels)
        all_probs.append(torch.ones(len(gt_labels), 81)/81.)
        continue
    probs = preds[:, 0:80]
    sums = probs.sum(dim=-1).unsqueeze(-1)
    bg_probs = torch.ones_like(sums) - sums
    probs = torch.cat([probs, bg_probs], dim=-1)
    is_bg = (torch.argmax(probs, dim=-1) == (probs.shape[1] - 1))
    max_probs, _ = probs.max(-1)
    is_conf = max_probs >= 0.
    mask = ~is_bg & is_conf
    probs = probs[mask]
    
    bbox_preds = preds[:, -5:-1]
    bbox_preds = bbox_preds[mask]
    
    if len(bbox_preds) == 0:
        count_no_preds += 1
        all_labels.append(gt_labels)
        all_probs.append(torch.ones(len(gt_labels), 81)/81.)
        continue
        
    #scale and convert bbox preds to match gt format
    #H, W, _ = sample['ori_shape']
    #factor = torch.tensor([W, H, W, H]).unsqueeze(0)
    #bbox_preds = torch.from_numpy(sample['bbox_preds'][-1]) 
    #bbox_preds = bbox_cxcywh_to_xyxy(bbox_preds) * factor
    
    #compute pairwise iou between all predictions and gt
    #matrix has shape N_preds x N_gt (for detr, N_preds == 100)
    ious = BboxOverlaps2D()(bbox_preds, gt_bboxes)
    
    #set iou to 0 for all boxes predicted as background (optional?)
    #bg_mask = probs[:, -1] >= 0.5 
    #ious[bg_mask] = 0
    
    #run min cost assignment algo using negative iou 
    #matrix has shape N_gt x 2
    matches = linear_assignment(1-ious.numpy())
    sel_indices = ious.numpy()[matches.numpy()[:, 0], matches.numpy()[:, 1]] >= IOU_THRESHOLD
    matches = matches[sel_indices]
    
    
    if len(matches) == 0:
        count_no_preds += 1
        all_labels.append(gt_labels)
        all_probs.append(torch.ones(len(gt_labels), 81)/81.)
        count_unmatched += len(gt_labels)
        continue
    
    #all gt boxes should be matched to something (could be background)
    if len(matches) != len(gt_bboxes):
        count_unmatched += len(gt_bboxes) - len(matches)
        matched_label_targets = torch.ones(len(matches)).long() * 80
        matched_label_targets = gt_labels[matches[:, 1]]
        all_probs.append(probs[matches[:, 0]])
        all_labels.append(matched_label_targets)
        unmatched_gt_indxs = np.setdiff1d(np.arange(len(gt_bboxes)), matches[:, 1])
        if len(unmatched_gt_indxs) != 0:
            all_labels.append(gt_labels[unmatched_gt_indxs])
            all_probs.append(torch.ones(len(unmatched_gt_indxs), 81)/81.)
        unmatched_pred_indxs = np.setdiff1d(np.arange(len(probs)), matches[:, 0])
        if len(unmatched_pred_indxs) != 0:
            all_labels.append(torch.ones(len(unmatched_pred_indxs)).long() * 80)
            all_probs.append(probs[torch.from_numpy(unmatched_pred_indxs)])
        continue
        #print(len(matches), len(gt_bboxes), len(bbox_preds), len(gt_bboxes))
    
    #convert full label targets
    #most predictions are unmatched and are therefore background
    label_targets = torch.ones(len(bbox_preds)).long() * 80 #background
    label_targets[matches[:, 0]] = gt_labels #gt_labels for matched predictions
    
    #save all probs and assigned labels
    all_probs.append(probs)
    all_labels.append(label_targets)

    
all_probs = torch.cat(all_probs)
all_labels = torch.cat(all_labels)
print(all_probs.shape, all_labels.shape)
count_unmatched, count_no_preds

torch.Size([70359, 81]) torch.Size([70359])


(12405, 37)

In [26]:
def get_nll(probs, labels):
    nll_vals = []
    for i in range(len(probs)):
        p = probs[i]
        l = labels[i]
        nll_vals.append(-torch.log(p[l]))
    nll_vals = torch.tensor(nll_vals)
    return nll_vals.mean().item()

def get_acc(probs, labels):
    _, max_idx = probs.max(-1)
    acc = (max_idx == labels).float().mean()
    return acc.item()

In [27]:
is_bg = all_labels == 80 
ece = _get_ece(all_probs[~is_bg].numpy(), all_labels[~is_bg].numpy())
nll = get_nll(all_probs[~is_bg], all_labels[~is_bg])
acc = get_acc(all_probs[~is_bg], all_labels[~is_bg])

ece, nll, acc

(0.11123149056928197, 1.68546724319458, 0.7030880451202393)

In [28]:
ece = _get_ece(all_probs[is_bg].numpy(), all_labels[is_bg].numpy())
nll = get_nll(all_probs[is_bg], all_labels[is_bg])
acc = get_acc(all_probs[is_bg], all_labels[is_bg])

ece, nll, acc

(0.7188036685695901, 1.6288549900054932, 0.0)

In [29]:
ece = _get_ece(all_probs.numpy(), all_labels.numpy())
nll = get_nll(all_probs, all_labels)
acc = get_acc(all_probs, all_labels)

ece, nll, acc

(0.3653759829108894, 1.6580899953842163, 0.363080769777298)

In [ ]:
#extract probability predicted for correct class
corr_probs = torch.tensor([all_probs[i, lt] for i, lt in enumerate(all_labels)])

#mask of locations assigned to background
is_bg = all_labels == 80 

### Across all boxes assigned to non-background, what is the classification accuracy?

In [ ]:
vals, idx = all_probs[~is_bg].max(dim=-1)
(idx == all_labels[~is_bg]).float().mean().item()

### Across all boxes assigned to background, what is the classification accuracy?

In [ ]:
vals, idx = all_probs[is_bg].max(dim=-1)
(idx == all_labels[is_bg]).float().mean().item()

### Across all boxes, what is the classification accuracy?

In [ ]:
vals, idx = all_probs.max(dim=-1)
(idx == all_labels).float().mean().item()

### Of all the boxes assigned to non-background, what precent predict background with the highest probability?

In [ ]:
vals, idx = all_probs[~is_bg].max(dim=-1)
(idx == 80).float().mean().item()

### Of all the boxes assigned to background, what precent predict background with the highest probability?

In [ ]:
vals, idx = all_probs[is_bg].max(dim=-1)
(idx == 80).float().mean().item()

### For all boxes assigned to non-background, what is the histogram of the probability of the most likely class?

In [ ]:
vals, idx = all_probs[~is_bg].max(dim=-1)
plt.hist(vals.numpy(), color='blue', bins=100, density=False);
plt.xlabel('Predicted Probability');
plt.ylabel('Count');

### For all boxes assigned to non-background, what is the histogram of the probability of the correct class?

In [ ]:
plt.hist(corr_probs[~is_bg].numpy(), color='blue', bins=100, density=False);
plt.xlabel('Predicted Probability');
plt.ylabel('Count');

### For all boxes assigned to background, what is the histogram of the probability of the most likely class?

In [ ]:
vals, idx = all_probs[is_bg].max(dim=-1)
plt.hist(vals.numpy(), color='blue', bins=100, density=False);
plt.xlabel('Predicted Probability');
plt.ylabel('Count');

### For all boxes assigned to background, what is the histogram of the probability of the correct class?

In [ ]:
vals, idx = all_probs[is_bg].max(dim=-1)
plt.hist(corr_probs[is_bg].numpy(), color='blue', bins=100, density=False);
plt.xlabel('Predicted Probability');
plt.ylabel('Count');